
# Assignment 4 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


    This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `Corona_NLP_train.csv` that you can get from the course website [git](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/Corona_NLP_train.csv). The data is "Coronavirus tweets NLP - Text Classification" that can be found on [kaggle](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification). The data has several columns, but we will only be working with `OriginalTweet`and `Sentiment`.

1. [3p] Load the data and filter out those tweets that have `Sentiment`=`Neutral`. Let $X$ represent the `OriginalTweet` and let 
    $$
        Y = 
        \begin{cases}
        1 & \text{if sentiment is towards positive}
        \\
        0 & \text{if sentiment is towards negative}.
        \end{cases}
    $$
    Put the resulting arrays into the variables $X$ and $Y$. Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, this is to make sure that we all did the same splits (we are in this case assuming the data is IID as presented in the dataset). That is [train,test,validation] is the splitting layout.

2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**
3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.
4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [1]:

# Part 1

# Load the data from the file specified in the problem definition and make sure that it is loaded using
# the search path `data/Corona_NLP_train.csv`. This is to make sure the autograder and your computer have the same
# file path and can load the data correctly.

# Contrary to how many other problems are structured, this problem actually requires you to
# have X on the shape (n_samples, ) that is a 1-dimensional array. Otherwise it will cause a bunch
# of errors in the autograder or also in for instance CountVectorizer.

# Make sure that all your data is numpy arrays and not pandas dataframes or series.
import pandas as pd
import numpy as np
df = pd.read_csv('data/Corona_NLP_train.csv', encoding='latin')
df = df.loc[df['Sentiment'] != 'Neutral']
df['Sentiment'] = df['Sentiment'].apply(lambda x: 1 if 'Positive' in x else 0)

X = df['OriginalTweet'].to_numpy()
Y = df['Sentiment'].to_numpy()

X_train = X[:20066]
Y_train = Y[:20066]
X_test = X[20066:25082]
Y_test = Y[20066:25082]
X_valid = X[25082:]
Y_valid = Y[25082:]



In [2]:

# Part 2

# Train a machine learning model or pipeline that can take the raw strings from X and predict Y=0,1 depending on the
# sentiment of the tweet. Store the trained model in the variable `model`.
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer 
model = Pipeline([('count_vectorizer', CountVectorizer()), ('logReg', LogisticRegression(solver='liblinear'))]).fit(X_train, Y_train)

model.score(X_test, Y_test)

0.8650318979266348

In [3]:
model.predict_proba(X_test)

array([[0.87766221, 0.12233779],
       [0.99712735, 0.00287265],
       [0.03467975, 0.96532025],
       ...,
       [0.58802982, 0.41197018],
       [0.94841932, 0.05158068],
       [0.92856718, 0.07143282]])

In [4]:

# Part 3

# Evaluate the model on the test set and calculate precision, and recall on both classes. Store the results in the
# variables `precision_0`, `precision_1`, `recall_0`, `recall_1`.
from sklearn.metrics import precision_score, recall_score
predictions = model.predict(X_test)
fp_1 = np.mean([1 if prediction==1 and Y_test[i]==0 else 0 for (i, prediction) in enumerate(predictions)])
fp_0 = np.mean([1 if prediction==0 and Y_test[i]==1 else 0 for (i, prediction) in enumerate(predictions)])
tp_1 = np.mean([1 if prediction==1 and Y_test[i]==1 else 0 for (i, prediction) in enumerate(predictions)])
tp_0 = np.mean([1 if prediction==0 and Y_test[i]==0 else 0 for (i, prediction) in enumerate(predictions)])

precision_0 = precision_score(Y_test, predictions, pos_label=0)
precision_1 = precision_score(Y_test, predictions, pos_label=1)
print(precision_0, precision_1)
print(tp_0/(tp_0+fp_0))

recall_0 = recall_score(Y_test, predictions, pos_label=0)
recall_1 = recall_score(Y_test, predictions, pos_label=1)
print(recall_0, recall_1)
print(tp_0/(tp_0+fp_1))

0.8579595426561126 0.8708971553610503
0.8579595426561126
0.8464208242950109 0.8808557727775729
0.8464208242950109


In [5]:

# Part 4

def cost(model,threshold,X,Y):
    # Hint, make sure that the model has a predict_proba method
    # think about how the decision is made based on the probabilities
    # and how the threshold can be used to make the decision.
    # For reference take a look at the lecture notes "Bayes classifier"
    # which contains how the decision is made based on the probabilities when the threshold is 0.5.
    
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost
    total_cost = 0
    probabilities = model.predict_proba(X)
    predictions = [1 if prob[1] > threshold else 0 for prob in probabilities]
    for (i, pred) in enumerate(predictions):
        if pred==1 and Y[i]==0:
            total_cost += 5
        if pred==0 and Y[i]==1:
            total_cost+=1

    return total_cost/len(X)

In [6]:

# Part 5

# Find the optimal threshold for the model on the test set. Store the threshold in the variable `optimal_threshold`
# and the cost at the optimal threshold in the variable `cost_at_optimal_threshold` evaluated on the test set.

thresholds = np.linspace(0, 1, 101)
costs = [cost(model, t, X_test, Y_test) for t in thresholds]
optimal_threshold = thresholds[np.argmin(costs)]
cost_at_optimal_threshold = cost(model, optimal_threshold, X_test, Y_test)

# from scipy.optimize import minimize

# def objective(threshold):
#     return cost(model, threshold, X_test, Y_test)

# result = minimize(objective, x0=0.3, method='trust-constr', bounds=[(0, 1)])
# optimal_threshold = result.x[0]
# cost_at_optimal_threshold = cost(model, optimal_threshold, X_test, Y_test)
# optimal_threshold

In [7]:
print(optimal_threshold, cost_at_optimal_threshold)
print(costs)

0.8 0.2727272727272727
[2.297647527910686, 1.9675039872408293, 1.8137958532695375, 1.7196969696969697, 1.6138357256778308, 1.541866028708134, 1.4736842105263157, 1.420255183413078, 1.3672248803827751, 1.3293460925039873, 1.2822966507177034, 1.2414274322169059, 1.2051435406698565, 1.157695374800638, 1.121212121212121, 1.081140350877193, 1.055023923444976, 1.0251196172248804, 0.9808612440191388, 0.9455741626794258, 0.9262360446570973, 0.8939393939393939, 0.8726076555023924, 0.8572567783094099, 0.8379186602870813, 0.8201754385964912, 0.7976475279106858, 0.7699362041467305, 0.7422248803827751, 0.720494417862839, 0.7077352472089314, 0.6862041467304625, 0.6684609250398724, 0.6425438596491229, 0.6251993620414673, 0.613237639553429, 0.595494417862839, 0.585127591706539, 0.5661881977671451, 0.5534290271132376, 0.5422647527910686, 0.527511961722488, 0.5105661881977671, 0.49262360446570974, 0.47607655502392343, 0.46850079744816586, 0.46351674641148327, 0.45075757575757575, 0.4368022328548644, 0.4

In [8]:

# Part 6

cost_at_optimal_threshold_valid = cost(model, optimal_threshold, X_valid, Y_valid)
deviation = np.sqrt((5**2*np.log(200))/(2*len(X_valid)))
cost_interval_valid = (cost_at_optimal_threshold_valid-deviation, cost_at_optimal_threshold_valid+deviation)

assert(type(cost_interval_valid) == tuple)
assert(len(cost_interval_valid) == 2)

In [9]:

# Part 7
costs = np.zeros(len(X_valid))
probabilities = model.predict_proba(X_valid)
predictions = [1 if prob[1] > optimal_threshold else 0 for prob in probabilities]
for (i, pred) in enumerate(predictions):
    if pred==1 and Y_valid[i]==0:
        costs[i] = 5
    if pred==0 and Y_valid[i]==1:
        costs[i] = 1

# Taken from the Concentration and limits notebook
def bennett_epsilon(n,b,sigma,alpha):
    import scipy.optimize as so
    h = lambda u: (1+u)*np.log(1+u)-u
    f = lambda epsilon: np.exp(-n*sigma**2/b**2*h(b*epsilon/sigma**2))-alpha/2
    ans = so.fsolve(f,0.002)
    epsilon = np.abs(ans[0])
    return epsilon



variance_of_C = np.var(costs)

epsilon = bennett_epsilon(len(X_valid), 5, variance_of_C, 0.01)
interval_of_C = (cost_at_optimal_threshold_valid-epsilon, cost_at_optimal_threshold_valid+epsilon)

assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)

In [10]:
print(variance_of_C)

0.6692569047766288
